<a href="https://colab.research.google.com/github/sooonsyk/Pocari/blob/main/classification/stoolnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### **base model**

In [ ]:
import os                                               #파일 접근
import torch                                            #신경망 사용
import torch.nn as nn
import numpy as np
import torch.optim as optim                             #최적화 알고리즘 제공
from torchvision import datasets, transforms            #데이터셋 사용, 이미지 데이터 전처리
from torch.utils.data import DataLoader                 #데이터 미니배치로 나누어서 로드
from sklearn.model_selection import train_test_split    #학습, 검증, 테스트 데이터 분리


# 이미지 전처리 및 데이터 증강
transform = transforms.Compose([
    transforms.Resize((200, 200)),        # 이미지 크기 조정
    transforms.ToTensor(),                # 이미지를 텐서로 변환
    transforms.Normalize((0.5,), (0.5,))  # 정규화
])

# 이미지 데이터셋 로드
dataset = datasets.ImageFolder(root='/content/drive/MyDrive/images_0601', transform=transform)

# 데이터셋을 학습, 검증, 테스트로 분할
train_val_indices, test_indices = train_test_split(list(range(len(dataset))), test_size=0.2, random_state=42)
train_indices, val_indices = train_test_split(train_val_indices, test_size=0.25, random_state=42)  # train 60%, val 20%, test 20%

train_dataset = torch.utils.data.Subset(dataset, train_indices)
val_dataset = torch.utils.data.Subset(dataset, val_indices)
test_dataset = torch.utils.data.Subset(dataset, test_indices)

# DataLoader 생성 - data 수 적어서 batch 지정 안 함
train_loader = DataLoader(dataset=train_dataset, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, shuffle=False)
test_loader = DataLoader(dataset=test_dataset, shuffle=False)

# 데이터셋 클래스 확인
print("Classes:", dataset.classes)
print("Number of training samples:", len(train_dataset))
print("Number of validation samples:", len(val_dataset))
print("Number of test samples:", len(test_dataset))

Classes: ['blood', 'diarrhea', 'green', 'normal', 'white']
Number of training samples: 27
Number of validation samples: 10
Number of test samples: 10


In [ ]:
class StoolNet(nn.Module):
    def __init__(self):
        super(StoolNet, self).__init__()
        # Convolutional layers
        self.conv1 = nn.Conv2d(3, 32, kernel_size=5, stride=1, padding=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2)
        # Max pooling layers
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        # Fully connected layers
        self.fc1 = nn.Linear(50 * 50 * 64, 512)
        self.fc2 = nn.Linear(512, 5)  # 클래스 수가 5개이므로 출력 크기는 5로 설정
        # Dropout layer
        self.dropout = nn.Dropout(p=0.5)

    def forward(self, x):
        # Convolutional layers with ReLU activation and max pooling
        x = torch.relu(self.conv1(x))
        x = self.pool(x)
        x = torch.relu(self.conv2(x))
        x = self.pool(x)
        # 3D 텐서 1D 텐서로 변환
        x = x.view(-1, 50 * 50 * 64)
        # Fully connected layers with ReLU activation and dropout
        x = self.dropout(torch.relu(self.fc1(x)))
        # 결과 출력
        x = self.fc2(x)
        return x


# StoolNet 모델 인스턴스 생성
model = StoolNet()

# 모델 구조 확인
print(model)

StoolNet(
  (conv1): Conv2d(3, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=160000, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=5, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)


In [ ]:
# 손실 함수 및 옵티마이저 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# 학습 함수 정의
def train(model, train_loader, val_loader, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        # 학습 단계
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)

        # 검증 단계
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * inputs.size(0)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        # 결과 출력
        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_val_loss = val_loss / len(val_loader.dataset)
        val_accuracy = correct / total
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Val Loss: {epoch_val_loss:.4f}, Val Acc: {val_accuracy:.4f}')

# StoolNet 모델 학습
train(model, train_loader, val_loader, criterion, optimizer, num_epochs=10)


Epoch [1/10], Loss: 1.6429, Val Loss: 1.4111, Val Acc: 0.6000
Epoch [2/10], Loss: 1.2948, Val Loss: 1.7223, Val Acc: 0.3000
Epoch [3/10], Loss: 0.9778, Val Loss: 2.5036, Val Acc: 0.6000
Epoch [4/10], Loss: 0.7180, Val Loss: 2.4721, Val Acc: 0.3000
Epoch [5/10], Loss: 0.7957, Val Loss: 2.3230, Val Acc: 0.5000
Epoch [6/10], Loss: 1.0851, Val Loss: 1.5753, Val Acc: 0.5000
Epoch [7/10], Loss: 0.9966, Val Loss: 2.4621, Val Acc: 0.2000
Epoch [8/10], Loss: 0.4746, Val Loss: 6.0247, Val Acc: 0.4000
Epoch [9/10], Loss: 0.3013, Val Loss: 5.8727, Val Acc: 0.3000
Epoch [10/10], Loss: 1.2166, Val Loss: 2.6068, Val Acc: 0.7000


In [ ]:
# 테스트 데이터셋을 사용하여 모델 평가
def test(model, test_loader):
    model.eval()
    correct = 0
    total = 0

    predicted_labels = []
    true_labels = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)

            # 예측된 레이블과 실제 레이블 저장
            predicted_labels.extend(predicted.tolist())
            true_labels.extend(labels.tolist())

            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    test_accuracy = correct / total
    print(f'Test Accuracy: {test_accuracy:.2%}')
    return true_labels, predicted_labels

true_labels, predicted_labels = test(model, test_loader)

# 실제 레이블과 예측된 레이블 출력
print("True Labels:", true_labels)
print("Predicted Labels:", predicted_labels)

Test Accuracy: 60.00%
True Labels: [3, 4, 3, 4, 3, 4, 2, 3, 1, 3]
Predicted Labels: [3, 4, 3, 4, 2, 3, 3, 3, 3, 3]


### **White balance 추가**

In [ ]:
import os
import torch
import torch.nn as nn
import numpy as np
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from skimage import img_as_ubyte
from PIL import Image

def white_patch(image, percentile=100):

    """
    Parameters
    ----------
    image : 입력으로 받는 이미지는 RGB 채널을 각각 가지는 (height, width, channels) 3차원 numpy 배열
    percentile : 채널 값의 보정값으로 사용하고자 하는 값을 지정하기 위한 비율, 기본값은 100으로 최대값을 사용
    """

    image = np.array(image)

    white_patch_image = img_as_ubyte(
        (image * 1.0 / np.percentile(image,
                                     percentile,
                                     axis=(0, 1))).clip(0, 1))
    return Image.fromarray(white_patch_image)


transform = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.Lambda(lambda x: white_patch(x, percentile=85)),  # 전처리 함수 적용
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# 이미지 데이터셋 로드
dataset = datasets.ImageFolder(root='/content/drive/MyDrive/images_0601', transform=transform)

# 데이터셋을 학습, 검증, 테스트로 분할
train_val_indices, test_indices = train_test_split(list(range(len(dataset))), test_size=0.2, random_state=42)
train_indices, val_indices = train_test_split(train_val_indices, test_size=0.25, random_state=42)  # train 60%, val 20%, test 20%

train_dataset = torch.utils.data.Subset(dataset, train_indices)
val_dataset = torch.utils.data.Subset(dataset, val_indices)
test_dataset = torch.utils.data.Subset(dataset, test_indices)

# DataLoader 생성 - data 수 적어서 batch 지정 안 함
train_loader = DataLoader(dataset=train_dataset, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, shuffle=False)
test_loader = DataLoader(dataset=test_dataset, shuffle=False)

# 데이터셋 클래스 확인
print("Classes:", dataset.classes)
print("Number of training samples:", len(train_dataset))
print("Number of validation samples:", len(val_dataset))
print("Number of test samples:", len(test_dataset))

Classes: ['blood', 'diarrhea', 'green', 'normal', 'white']
Number of training samples: 27
Number of validation samples: 10
Number of test samples: 10


In [ ]:
class StoolNet(nn.Module):
    def __init__(self):
        super(StoolNet, self).__init__()
        # Convolutional layers
        self.conv1 = nn.Conv2d(3, 32, kernel_size=5, stride=1, padding=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2)
        # Max pooling layers
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        # Fully connected layers
        self.fc1 = nn.Linear(50 * 50 * 64, 512)
        self.fc2 = nn.Linear(512, 5)  # 클래스 수가 5개이므로 출력 크기는 5로 설정
        # Dropout layer
        self.dropout = nn.Dropout(p=0.5)

    def forward(self, x):
        # Convolutional layers with ReLU activation and max pooling
        x = torch.relu(self.conv1(x))
        x = self.pool(x)
        x = torch.relu(self.conv2(x))
        x = self.pool(x)
        # 3D 텐서 1D 텐서로 변환
        x = x.view(-1, 50 * 50 * 64)
        # Fully connected layers with ReLU activation and dropout
        x = self.dropout(torch.relu(self.fc1(x)))
        # 결과 출력
        x = self.fc2(x)
        return x

In [ ]:
# 손실 함수 및 옵티마이저 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# 학습 함수 정의
def train(model, train_loader, val_loader, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        # 학습 단계
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)

        # 검증 단계
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * inputs.size(0)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        # 결과 출력
        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_val_loss = val_loss / len(val_loader.dataset)
        val_accuracy = correct / total
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Val Loss: {epoch_val_loss:.4f}, Val Acc: {val_accuracy:.4f}')

# StoolNet 모델 학습
train(model, train_loader, val_loader, criterion, optimizer, num_epochs=10)


Epoch [1/10], Loss: 0.6078, Val Loss: 8.8295, Val Acc: 0.2000
Epoch [2/10], Loss: 0.2956, Val Loss: 6.4987, Val Acc: 0.4000
Epoch [3/10], Loss: 0.9034, Val Loss: 4.0934, Val Acc: 0.6000
Epoch [4/10], Loss: 4.9964, Val Loss: 2.5359, Val Acc: 0.0000
Epoch [5/10], Loss: 1.6561, Val Loss: 2.1224, Val Acc: 0.2000
Epoch [6/10], Loss: 6.1548, Val Loss: 1.8241, Val Acc: 0.1000
Epoch [7/10], Loss: 1.5156, Val Loss: 1.4686, Val Acc: 0.6000
Epoch [8/10], Loss: 1.4155, Val Loss: 1.6988, Val Acc: 0.3000
Epoch [9/10], Loss: 1.7199, Val Loss: 1.3363, Val Acc: 0.6000
Epoch [10/10], Loss: 0.9629, Val Loss: 2.2830, Val Acc: 0.6000


In [ ]:
# 테스트 데이터셋을 사용하여 모델 평가
def test(model, test_loader):
    model.eval()
    correct = 0
    total = 0

    predicted_labels = []
    true_labels = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)

            # 예측된 레이블과 실제 레이블 저장
            predicted_labels.extend(predicted.tolist())
            true_labels.extend(labels.tolist())

            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    test_accuracy = correct / total
    print(f'Test Accuracy: {test_accuracy:.2%}')
    return true_labels, predicted_labels

true_labels, predicted_labels = test(model, test_loader)

# 실제 레이블과 예측된 레이블 출력
print("True Labels:", true_labels)
print("Predicted Labels:", predicted_labels)

Test Accuracy: 40.00%
True Labels: [3, 4, 3, 4, 3, 4, 2, 3, 1, 3]
Predicted Labels: [3, 3, 3, 3, 2, 3, 3, 3, 3, 3]


### **이미지 증강 추가**

In [ ]:
import os
import torch
import torch.nn as nn
import numpy as np
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from skimage import img_as_ubyte
from PIL import Image

def white_patch(image, percentile=100):
    """
    Returns a plot comparison of original and corrected/white balanced image
    using the White Patch algorithm.

    Parameters
    ----------
    image : numpy array
            Image to process using white patch algorithm
    percentile : integer, optional
                  Percentile value to consider as channel maximum
    """
    image = np.array(image)

    white_patch_image = img_as_ubyte(
        (image * 1.0 / np.percentile(image,
                                     percentile,
                                     axis=(0, 1))).clip(0, 1))
    return Image.fromarray(white_patch_image)

transform = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.Lambda(lambda x: white_patch(x, percentile=85)),  # 전처리 함수 적용
    transforms.RandomHorizontalFlip(),  # 랜덤으로 이미지를 수평으로 뒤집기
    transforms.RandomVerticalFlip(p=0.5), # 랜덤으로 이미지를 수직으로 뒤집기
    transforms.RandomRotation(degrees=(-10, 10)), # 랜덤으로 이미지 회전
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # 정규화
])

# 이미지 데이터셋 로드
dataset = datasets.ImageFolder(root='/content/drive/MyDrive/images_0601', transform=transform)

# 데이터셋을 학습, 검증, 테스트로 분할
train_val_indices, test_indices = train_test_split(list(range(len(dataset))), test_size=0.2, random_state=42)
train_indices, val_indices = train_test_split(train_val_indices, test_size=0.25, random_state=42)  # train 60%, val 20%, test 20%

train_dataset = torch.utils.data.Subset(dataset, train_indices)
val_dataset = torch.utils.data.Subset(dataset, val_indices)
test_dataset = torch.utils.data.Subset(dataset, test_indices)

# DataLoader 생성 - data 수 적어서 batch 지정 안 함
train_loader = DataLoader(dataset=train_dataset, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, shuffle=False)
test_loader = DataLoader(dataset=test_dataset, shuffle=False)

# 데이터셋 클래스 확인
print("Classes:", dataset.classes)
print("Number of training samples:", len(train_dataset))
print("Number of validation samples:", len(val_dataset))
print("Number of test samples:", len(test_dataset))

Classes: ['blood', 'diarrhea', 'green', 'normal', 'white']
Number of training samples: 27
Number of validation samples: 10
Number of test samples: 10


In [ ]:
class StoolNet(nn.Module):
    def __init__(self):
        super(StoolNet, self).__init__()
        # Convolutional layers
        self.conv1 = nn.Conv2d(3, 32, kernel_size=5, stride=1, padding=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2)
        # Max pooling layers
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        # Fully connected layers
        self.fc1 = nn.Linear(50 * 50 * 64, 512)
        self.fc2 = nn.Linear(512, 5)  # 클래스 수가 5개이므로 출력 크기는 5로 설정
        # Dropout layer
        self.dropout = nn.Dropout(p=0.5)

    def forward(self, x):
        # Convolutional layers with ReLU activation and max pooling
        x = torch.relu(self.conv1(x))
        x = self.pool(x)
        x = torch.relu(self.conv2(x))
        x = self.pool(x)
        # 3D 텐서 1D 텐서로 변환
        x = x.view(-1, 50 * 50 * 64)
        # Fully connected layers with ReLU activation and dropout
        x = self.dropout(torch.relu(self.fc1(x)))
        # 결과 출력
        x = self.fc2(x)
        return x

In [ ]:
# 손실 함수 및 옵티마이저 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# 학습 함수 정의
def train(model, train_loader, val_loader, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        # 학습 단계
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)

        # 검증 단계
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * inputs.size(0)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        # 결과 출력
        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_val_loss = val_loss / len(val_loader.dataset)
        val_accuracy = correct / total
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Val Loss: {epoch_val_loss:.4f}, Val Acc: {val_accuracy:.4f}')

# StoolNet 모델 학습
train(model, train_loader, val_loader, criterion, optimizer, num_epochs=10)


Epoch [1/10], Loss: 1.7074, Val Loss: 1.1078, Val Acc: 0.6000
Epoch [2/10], Loss: 1.2558, Val Loss: 0.9307, Val Acc: 0.7000
Epoch [3/10], Loss: 1.2411, Val Loss: 1.0446, Val Acc: 0.7000
Epoch [4/10], Loss: 1.1634, Val Loss: 1.1637, Val Acc: 0.7000
Epoch [5/10], Loss: 1.1648, Val Loss: 0.9652, Val Acc: 0.7000
Epoch [6/10], Loss: 1.1105, Val Loss: 1.2308, Val Acc: 0.5000
Epoch [7/10], Loss: 1.0454, Val Loss: 1.4434, Val Acc: 0.3000
Epoch [8/10], Loss: 1.0926, Val Loss: 1.3187, Val Acc: 0.6000
Epoch [9/10], Loss: 0.8042, Val Loss: 1.5268, Val Acc: 0.6000
Epoch [10/10], Loss: 0.6870, Val Loss: 1.5043, Val Acc: 0.5000


In [ ]:
# 테스트 데이터셋을 사용하여 모델 평가
def test(model, test_loader):
    model.eval()
    correct = 0
    total = 0

    predicted_labels = []
    true_labels = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)

            # 예측된 레이블과 실제 레이블 저장
            predicted_labels.extend(predicted.tolist())
            true_labels.extend(labels.tolist())

            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    test_accuracy = correct / total
    print(f'Test Accuracy: {test_accuracy:.2%}')
    return true_labels, predicted_labels

true_labels, predicted_labels = test(model, test_loader)

# 실제 레이블과 예측된 레이블 출력
print("True Labels:", true_labels)
print("Predicted Labels:", predicted_labels)


Test Accuracy: 70.00%
True Labels: [3, 4, 3, 4, 3, 4, 2, 3, 1, 3]
Predicted Labels: [3, 4, 3, 4, 3, 4, 2, 2, 2, 2]
